# Weakly connected components

Weakly connected components was added in MADlib 1.12.

In [23]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [26]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [27]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rc/v1.9alpha-rc1-188-ge47dc8a, cmake configuration time: Wed Jul 5 18:23:50 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create vertex and edge tables

In [31]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
    id INTEGER
);

CREATE TABLE edge(
    src INTEGER,
    dest INTEGER,
    user_id INTEGER
);

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(10),
(11),
(12),
(13),
(14),
(15),
(16);

INSERT INTO edge VALUES
(0, 1, 1),
(0, 2, 1),
(1, 2, 1),
(1, 3, 1),
(2, 3, 1),
(2, 5, 1),
(2, 6, 1),
(3, 0, 1),
(5, 6, 1),
(6, 3, 1),
(10, 11, 2),
(10, 12, 2),
(11, 12, 2),
(11, 13, 2),
(12, 13, 2),
(13, 10, 2),
(15, 16, 2),
(15, 14, 2);

SELECT * FROM edge ORDER BY src, dest;

Done.
Done.
Done.
14 rows affected.
18 rows affected.
18 rows affected.


src,dest,user_id
0,1,1
0,2,1
1,2,1
1,3,1
2,3,1
2,5,1
2,6,1
3,0,1
5,6,1
6,3,1


# 2.  Find all weakly connected components

In [32]:
%%sql
DROP TABLE IF EXISTS wcc_out;

SELECT madlib.weakly_connected_components(
                         'vertex',             -- Vertex table
                         'id',                 -- Vertix id column
                         'edge',               -- Edge table
                         'src=src, dest=dest', -- Comma delimted string of edge arguments
                         'wcc_out');      -- Output table of weakly connected components

SELECT * FROM wcc_out ORDER BY component_id, id;

Done.
1 rows affected.
14 rows affected.


id,component_id
0,0
1,0
2,0
3,0
5,0
6,0
4,4
10,10
11,10
12,10


# 3.  Weakly connected components with grouping

In [33]:
%%sql
DROP TABLE IF EXISTS wcc_out, wcc_out_summary;

SELECT madlib.weakly_connected_components(
                         'vertex',             -- Vertex table
                         'id',                 -- Vertix id column
                         'edge',               -- Edge table
                         'src=src, dest=dest', -- Comma delimted string of edge arguments
                         'wcc_out',            -- Output table of weakly connected components
                         'user_id');           -- Grouping column name

SELECT * FROM wcc_out ORDER BY component_id, id, user_id;

Done.
1 rows affected.
13 rows affected.


id,component_id,user_id
0,0,1
1,0,1
2,0,1
3,0,1
5,0,1
6,0,1
10,10,2
11,10,2
12,10,2
13,10,2


Note that vertex '4' is not identified as a separate component in the above result. This is because disconnected nodes cannot be assigned to a particular group with the current graph representation in MADlib.